In [1156]:
# !pip install rdflib
# !pip install morph-kgc
# !pip install owlrl

In [1157]:
import morph_kgc
g = morph_kgc.materialize('../mappings/config.ini')

g.serialize(destination='resultado.nt', format='nt')
print("Archivo resultado.nt generado")

INFO | 2025-11-08 20:01:13,781 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2025-11-08 20:01:14,566 | 52 mapping rules retrieved.
INFO | 2025-11-08 20:01:14,582 | Mapping partition with 38 groups generated.
INFO | 2025-11-08 20:01:14,584 | Maximum number of rules within mapping group: 3.
INFO | 2025-11-08 20:01:14,585 | Mappings processed in 0.800 seconds.
INFO | 2025-11-08 20:01:15,120 | Number of triples generated in total: 9349.


Archivo resultado.nt generado


c:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\plugins\serializers\nt.py:41: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


In [1158]:
from rdflib import Graph
from owlrl import DeductiveClosure, RDFS_Semantics

g = Graph()

g.parse("../ontology/ontology.ttl", format="turtle")
g.parse("resultado.nt", format="nt")

DeductiveClosure(RDFS_Semantics).expand(g)

In [1159]:
PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#> 
PREFIX schema: <https://schema.org/> 
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#> 
PREFIX dbo:    <http://dbpedia.org/ontology/> 
PREFIX owl:    <http://www.w3.org/2002/07/owl#> 
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#> 
"""


print(f"Triples cargados: {len(g)}")

# Mostrar las primeras 10 tripletas
for i, (s, p, o) in enumerate(g):
    print(s, p, o)
    if i >= 5:
        break

Triples cargados: 21132
http://smartcity.linkeddata.es/lcc/resource/facility/Park/Parque-Emir-Mohamed-I http://smartcity.linkeddata.es/lcc/ontology#accessibility Instalación parcialmente accesible para personas con movilidad reducida
913603265 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2001/XMLSchema#string
http://smartcity.linkeddata.es/lcc/resource/facility/Park/Parque-de-Berlin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/District
http://smartcity.linkeddata.es/lcc/resource/bus/Bus%3A%20601 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://smartcity.linkeddata.es/lcc/ontology#Bus
http://smartcity.linkeddata.es/lcc/resource/facility/EducationalInstitution/Escuela-Tecnica-Superior-de-Ingenieros-de-Telecomunicacion http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://smartcity.linkeddata.es/lcc/resource/EducationalInstitution
913135528
913135531 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-sch

In [1160]:
print(f"Triples cargados: {len(g)}")

PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
"""

def qrun(q, limit=50):
    q = PREFIXES + q
    res = g.query(q)
    print(f"--- {len(res)} filas ---")
    for i,row in enumerate(res):
        print([str(x) if x is not None else None for x in row])
        if i >= limit-1: break

Triples cargados: 21132


In [1161]:
########################################################################################
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter
########################################################################################

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:Facility .
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type
""")

--- 8 filas ---
['Educational Institution', '117']
['Faculty', '35']
['Library', '50']
['Other Centers', '13']
['Park or Garden', '203']
['Residence Hall', '48']
['Sport Facility', '81']
['University School', '21']


In [1162]:
########################################################################################
# ✅ Número de individuos de cada subclase DIRECTA de Facility, no aparecen subclases de subclases 
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter
########################################################################################

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:Facility .
  
  # Esto asegura que ?subclass sea un hijo directo: no hay clase intermedia entre Facility y ?subclass
  FILTER NOT EXISTS {
    ?intermediate rdfs:subClassOf sc:Facility .
    ?subclass rdfs:subClassOf ?intermediate .
    FILTER(?intermediate != sc:Facility)
  }
  
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type

""")

--- 4 filas ---
['Educational Institution', '117']
['Library', '50']
['Park or Garden', '203']
['Sport Facility', '81']


In [1163]:
########################################################################################
# ✅ Número de individuos de cada subclase DIRECTA de Facility, no aparecen subclases de subclases 
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter
########################################################################################

for subtype,label in [
    ("sc:EducationalInstitution","EducationalInstitutions"),
    ("schema:Library","PublicLibraries"),
    ("schema:Park","ParksGardens"),
    ("sc:SportsFacility","SportsCenters")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {subtype} .

  # Filtrar solo subclases directas de Facility
  ?subclass rdfs:subClassOf sc:Facility .
  ?x a ?subclass .

  FILTER NOT EXISTS {{
    ?intermediate rdfs:subClassOf sc:Facility .
    ?subclass rdfs:subClassOf ?intermediate .
    FILTER(?intermediate != sc:Facility)
  }}

  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### EducationalInstitutions
--- 5 filas ---
['Colegio Mayor Ximénez de Cisneros']
['Residencia Universitaria Erasmo']
['Residencia Universitaria Gómez Pardo']
['Instituto de Estudios Bursátiles']
['Escuela Técnica Superior de Ingenieros de Telecomunicación']

### PublicLibraries
--- 5 filas ---
['Biblioteca Pública Municipal Gloria Fuertes (Barajas)']
['Biblioteca Pública Municipal Manuel Vázquez Montalbán (Tetuán)']
['Biblioteca Pública Municipal José Hierro (San Blas-Canillejas)']
['Biblioteca Pública Elena Fortún (Retiro)']
['Biblioteca Pública Municipal Pablo Neruda (Ciudad Lineal)']

### ParksGardens
--- 5 filas ---
['Jardín Botánico de Pradolongo']
['Parque Salvador de Madariaga']
['Parque de la Dehesa Boyal']
['Parque Alzola']
['Jardín Glorieta de Azorín']

### SportsCenters
--- 5 filas ---
['Instalación Deportiva Municipal Básica Victoria Hernández (El Espinillo II - centro deportivo)']
['Centro Deportivo Municipal Miguel Guillén Prim']
['Centro Deportivo Municipal Raúl Gonzál

In [1164]:
########################################################################################
# ✅ Tipos de Public Transport, suclases de public transport: (subwary, train, bus)
########################################################################################

qrun("""
SELECT ?type WHERE {
  ?x a ?subclass .
  ?subclass rdfs:subClassOf sc:PublicTransport .
  ?subclass rdfs:label ?type .
}
GROUP BY ?type
ORDER BY ?type
""")

--- 3 filas ---
['Bus']
['Subway']
['Train']


In [1165]:
########################################################################################
# Numero de facilities que tienen bus, metro o cercanias
########################################################################################

qrun("""
SELECT ?type (COUNT(?station) AS ?count) WHERE {
  ?station a ?subclass .
  ?subclass rdfs:subClassOf sc:PublicTransport .
  ?subclass rdfs:label ?type .
  
  # Que esté realmente asociado a alguna facility
  ?facility sc:hasNearby ?station .
}
GROUP BY ?type
ORDER BY ?type
""")

#Bus = 273 ; Subway = 329 Train: 85

--- 3 filas ---
['Bus', '273']
['Subway', '409']
['Train', '414']


In [1166]:
# Número de individuos de Metro, Bus y Tren (Cercanías)

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:Subway       BIND("Subway"        AS ?type) } UNION
  { ?x a sc:Bus          BIND("Bus"           AS ?type) } UNION
  { ?x a sc:Train        BIND("Train" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 3 filas ---
['Bus', '254']
['Subway', '309']
['Train', '314']


In [1167]:
########################################################################################
# ✅ 5 ejemplos de Estaciones de Metro y Tren 
########################################################################################

for ttype, label in [
    ("sc:Subway", "Metro Stations"),  # Cambiamos de SubwayStation a Subway
    ("sc:Train", "Train Stations")  # Cambiamos de TrainStation a Train
]:
    print(f"\n### {label}")
    # Modificamos la consulta para buscar el recurso de transporte (?t)
    qrun(f"""
SELECT DISTINCT ?stationName WHERE {{
  ?t a {ttype} .        
  ?t sc:hasStations ?stationName .
}}
LIMIT 5
""", limit=5)



### Metro Stations
--- 5 filas ---
["O'Donnell"]
['Buenos Aires']
['Tribunal, Alonso Martínez, Bilbao']
['Paco de Lucía']
['Conde de Casal']

### Train Stations
--- 5 filas ---
['Nuevos Ministerios']
['San Cristóbal']
['Las Águilas']
['Embajadores, Atocha']
['Laguna, Aluche']


In [1168]:
########################################################################################
# ✅ 5 ejemplos de Lineas de Metro, Tren y bus
########################################################################################

for ttype, label in [
    ("sc:Subway", "Metro lines"),  
    ("sc:Train", "Train lines"),  
    ("sc:Bus", "Bus lines") 
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT DISTINCT ?busLine WHERE {{
  ?t a {ttype} .        
  ?t sc:hasLines ?busLine .
}}
LIMIT 5
""", limit=5)


### Metro lines
--- 5 filas ---
['1']
['9']
['6']
['6\r\n7']
['2']

### Train lines
--- 5 filas ---
['C-1\r\nC-2\r\nC-7']
['C-1\r\nC-3\r\nC-4\r\nC-7\r\nC-8\r\nC-10']
['C-3']
['C-5']
['C-5\r\nC-1\r\nC-2\r\nC-3\r\nC-4\r\nC-6\r\nC-7\r\nC-8\r\nC-10']

### Bus lines
--- 5 filas ---
['601']
['49\r\n64']
['45']
['27\r\n40\r\n126\r\n147\r\n150']
['17\r\n36\r\n41\r\n62']


In [1169]:
#  Número de individuos de Municipio, Distrito y Barrio

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a dbo:Municipality  BIND("Municipality"  AS ?type) } UNION
  { ?x a dbo:District      BIND("District"      AS ?type) } UNION
  { ?x a dbo:Neighbourhood BIND("Neighbourhood" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")

--- 3 filas ---
['District', '856']
['Municipality', '856']
['Neighbourhood', '974']


In [ ]:
########################################################################################
# ✅  5 ejemplos de Municipio, Distrito y Barrio (solo nombre)
########################################################################################

for ctype,label in [
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood"),
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {ctype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Municipality
--- 5 filas ---
['MADRID']
['MADRID']
['MADRID']
['MADRID']
['MADRID']

### District
--- 5 filas ---
['USERA']
['TETUAN']
['MONCLOA-ARAVACA']
['CARABANCHEL']
['SAN BLAS-CANILLEJAS']

### Neighbourhood
--- 5 filas ---
['SAN PASCUAL']
['NUMANCIA']
['CASA DE CAMPO']
['NUEVA ESPAÑA']
['ENSANCHE DE VALLECAS']


In [ ]:
########################################################################################
# ✅  Comprobación jerarquía Municipio→Distrito→Barrio
########################################################################################

print("### Neigbourhood → District")
qrun("""
SELECT ?municipality ?district WHERE {
  ?m a dbo:Neighbourhood ; schema:name ?municipality ; sc:locatedInDistrict ?d .
  OPTIONAL { ?d schema:name ?district }
}
ORDER BY ?municipality ?district
""", limit=20)

print("\n### District → Municipality")
qrun("""
SELECT ?district ?neighbourhood WHERE {
  ?d a dbo:District ; schema:name ?district ; sc:locatedInMunicipality ?n .
  OPTIONAL { ?n schema:name ?neighbourhood }
}
ORDER BY ?district ?neighbourhood
""", limit=20)


### Neigbourhood → District
--- 119 filas ---
['ABRANTES', 'CARABANCHEL']
['ACACIAS', 'ARGANZUELA']
['ADELFAS', 'RETIRO']
['ALAMEDA DE OSUNA', 'BARAJAS']
['ALMAGRO', 'CHAMBERI']
['ALMENARA', 'TETUAN']
['ALMENDRALES', 'USERA']
['ALUCHE', 'LATINA']
['AMBROZ', 'VICALVARO']
['APOSTOL SANTIAGO', 'HORTALEZA']
['ARAPILES', 'CHAMBERI']
['ARAVACA', 'MONCLOA-ARAVACA']
['ARCOS', 'SAN BLAS-CANILLEJAS']
['ARGUELLES', 'MONCLOA-ARAVACA']
['ATALAYA', 'CIUDAD LINEAL']
['BELLAS VISTAS', 'TETUAN']
['BERRUGUETE', 'TETUAN']
['BUENAVISTA', 'CARABANCHEL']
['BUTARQUE', 'VILLAVERDE']
['CANILLAS', 'HORTALEZA']

### District → Municipality
--- 22 filas ---
['ARGANZUELA', 'MADRID']
['BARAJAS', 'MADRID']
['CARABANCHEL', 'MADRID']
['CENTRO', 'MADRID']
['CHAMARTIN', 'MADRID']
['CHAMBERI', 'MADRID']
['CIUDAD LINEAL', 'MADRID']
['DISTRITO', 'MADRID']
['FUENCARRAL-EL PARDO', 'MADRID']
['HORTALEZA', 'MADRID']
['LATINA', 'MADRID']
['MONCLOA-ARAVACA', 'MADRID']
['MORATALAZ', 'MADRID']
['PUENTE DE VALLECAS', 'MADRID']
['RE

In [ ]:
########################################################################################
# ✅  Comprobar relaciones subclases de Facility → Barrio
########################################################################################

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:Park","ParksGardens"),
    ("sc:SportsFacility","SportsCenters")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
BIND(COALESCE(?fraw, REPLACE(REPLACE(STR(?f), '^.*/', ''), '%20', ' ')) AS ?facility)
BIND(COALESCE(?nraw, REPLACE(REPLACE(STR(?n), '^.*/', ''), '%20', ' ')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)



### Universities → Neighbourhood
--- 5 filas ---
['Colegio Mayor Ximénez de Cisneros', 'CIUDAD UNIVERSITARIA']
['Residencia Universitaria Erasmo', 'EL GOLOSO']
['Residencia Universitaria Gómez Pardo', 'RIOS ROSAS']
['Instituto de Estudios Bursátiles', 'ARAVACA']
['Escuela Técnica Superior de Ingenieros de Telecomunicación', 'CIUDAD UNIVERSITARIA']

### PublicLibraries → Neighbourhood
--- 5 filas ---
['Biblioteca Pública Municipal Gloria Fuertes (Barajas)', 'CORRALEJOS']
['Biblioteca Pública Municipal Manuel Vázquez Montalbán (Tetuán)', 'BELLAS VISTAS']
['Biblioteca Pública Municipal José Hierro (San Blas-Canillejas)', 'ROSAS']
['Biblioteca Pública Elena Fortún (Retiro)', 'ADELFAS']
['Biblioteca Pública Municipal Pablo Neruda (Ciudad Lineal)', 'PUEBLO NUEVO']

### ParksGardens → Neighbourhood
--- 5 filas ---
['Jardín Botánico de Pradolongo', 'PRADOLONGO']
['Parque Salvador de Madariaga', 'SAN PASCUAL']
['Parque de la Dehesa Boyal', 'SAN CRISTOBAL']
['Parque Alzola', 'ORCASITAS']
['Jard

In [1173]:
# 🚈 Comprobar relaciones subclases de Facility → Metro/Bus/Tren
# - Para cada subclase: cuenta cuántas instalaciones tienen cada tipo de transporte cercano

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    
    print(f"\n### {label} con transporte cercano (conteos por tipo)")
    qrun(f"""
SELECT ?transportType (COUNT(DISTINCT ?f) AS ?n) WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?transportType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?transportType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?transportType)
  }}
}}
GROUP BY ?transportType
ORDER BY DESC(?n)
""", limit=10)

    print(f"### {label} → ejemplos (nombre facility, tipo transporte, nombre transporte)")
    qrun(f"""
SELECT ?facility ?tType ?tName WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?t schema:name ?tname }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?tname, REPLACE(STR(?t), '^.*/', '')) AS ?tName)
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?tType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?tType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?tType)
  }}
}}
LIMIT 5
""", limit=5)



### Universities con transporte cercano (conteos por tipo)
--- 1 filas ---
### Universities → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---

### PublicLibraries con transporte cercano (conteos por tipo)
--- 1 filas ---
### PublicLibraries → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---

### ParksGardens con transporte cercano (conteos por tipo)
--- 1 filas ---
### ParksGardens → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---

### SportsCenters con transporte cercano (conteos por tipo)
--- 1 filas ---
### SportsCenters → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 0 filas ---


In [1174]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:Facility   BIND("University"    AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")

--- 1 filas ---
['University', '856']


In [1175]:
# --------------------------------------------------------------------
query_2 = """
SELECT (COUNT(?park) AS ?total)  # Cambiamos ?count a ?total para evitar colisiones con el método
WHERE {
?park a schema:ParkGarden ;
        sc:accessibility "Instalación parcialmente accesible para personas con movilidad reducida" .
}
"""

print("\n--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---")
try:
    results_2 = g.query(PREFIXES + query_2)
    
    # COUNT devuelve una única fila. Iteramos sobre ella y accedemos al valor.
    for row in results_2:

        count_value = row.total.value if hasattr(row.total, 'value') else row.total
        print(f"Total de Parques Parcialmente Accesibles: {count_value}")

        
except Exception as e:
    print(f"Error al ejecutar la Consulta 2: {e}")


--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---
Total de Parques Parcialmente Accesibles: 0


In [1181]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:Facility","Facilities")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
BIND(COALESCE(?fraw, REPLACE(REPLACE(STR(?f), '^.*/', ''), '%20', ' ')) AS ?facility)
BIND(COALESCE(?nraw, REPLACE(REPLACE(STR(?n), '^.*/', ''), '%20', ' ')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)


### Facilities → Neighbourhood
--- 5 filas ---
['Biblioteca Pública Municipal Francisco Ayala (Vicálvaro)', 'VALDEBERNARDO']
['Parque de Ana Tutor', 'PE%C3%91A GRANDE']
['Universidad Complutense de Madrid. Facultad de Comercio y Turismo', 'VALLEHERMOSO']
['Colegio Mayor Universitario Isabel de España', 'CIUDAD UNIVERSITARIA']
['Parque de Arriaga', 'PUEBLO NUEVO']
